# Notebook for cleanning, EDA, and Model Development
---
*By Patcharanat P.*

## Load Data

In [ ]:
import numpy as np
import pandas as pd
# import requests
# import io

In [ ]:
# response = requests.get("https://github.com/Patcharanat/GrabSpark-Data-Analytics-for-Business-Use-Cases/blob/master/sample_a.csv")
# pd.read_csv(response.content)
# pd.read_csv(io.StringIO(response.content.decode('utf-8')))

# must use raw file
# response = requests.get("https://raw.githubusercontent.com/Patcharanat/ecommerce-invoice/master/data/cleaned_data.csv")
# content = response.text
# print(content)

## Data Cleaning

In [ ]:
# df = pd.read_csv("./data/uncleaned_data.csv", encoding='cp1252')
# df = pd.read_csv("./data/uncleaned_data.csv")
# df = pd.read_csv("./data/data_url_uncleaned.csv", low_memory=False)
# df = pd.read_csv("./data/data_api_uncleaned.csv", encoding='cp1252')
# df = pd.read_csv("./data/data_postgres_cleaned.csv", encoding='cp1252')
# df = pd.read_csv("./data/data_postgres_cleaned.csv")
# df = pd.read_csv("./data/data_api_uncleaned.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/Patcharanat/ecommerce-invoice/master/data/cleaned_data.csv")
df = pd.read_parquet("./data/ecomm_invoice_transaction.parquet")
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.loc[df['Description'].isnull()].head()
# df.loc[df['Description'].isnull() & df['CustomerID'].isnull()]

In [ ]:
df.loc[df['CustomerID'].isnull()].head()

In [ ]:
df.loc[df['UnitPrice'] < 0]

In [ ]:
df.loc[df['Quantity'] < 0].head(10)

In [ ]:
df.loc[df['StockCode'] == 'm']

In [ ]:
df.loc[df['InvoiceNo'] == 'C536379']

In [ ]:
df.plot(kind='box')

### Fill NA

In [ ]:
df_temp = df.copy()

In [ ]:
df_temp['Description'] = df['Description'].fillna('No Description')

In [ ]:
df_temp['CustomerID'] = df['CustomerID'].fillna(0)
df_temp['CustomerID'] = df_temp['CustomerID'].astype(float).astype(int)

In [ ]:
df_temp['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
df_temp.isnull().sum()

In [ ]:
df_temp.head()

In [ ]:
df_temp['InvoiceNo'] = df_temp['InvoiceNo'].astype(str)
df_temp['StockCode'] = df_temp['StockCode'].astype(str)
df_temp['Description'] = df_temp['Description'].astype(str)
# df_temp['Quantity'] = df_temp['Quantity']
# df_temp['InvoiceDate'] = df_temp['InvoiceDate']
# df_temp['UnitPrice'] = df_temp['UnitPrice'].astype('float64')
df_temp['Country'] = df_temp['Country'].astype(str)

In [ ]:
df_temp.info()

In [ ]:
# df_temp.to_csv("./data/cleaned_data.csv", index=False)

## EDA

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# sns.set(style="dark")
plt.style.use("ggplot")

In [ ]:
df_temp = pd.read_csv("./data/cleaned_data.csv")

In [ ]:
df_temp.info()

In [ ]:
df_temp["total_spend"] = df_temp["Quantity"] * df_temp["UnitPrice"]
df_temp.head()

In [ ]:
x = df_temp.groupby(by=['InvoiceDate'])['total_spend'].sum().sort_index(ascending=True).index
y = df_temp.groupby(by=['InvoiceDate'])['total_spend'].sum().sort_index(ascending=True).values

In [ ]:
timeseries_data = pd.DataFrame({'invoice_date': x, 'total_spend': y})
timeseries_data.head()

In [ ]:
# sns.lineplot(x=x, y=y)
# timeseries_data['invoice_date'] = 
# sample = timeseries_data.set_index('invoice_date')
tick = ['2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12']
sample = timeseries_data.set_index(pd.to_datetime(timeseries_data['invoice_date'], format="%Y-%m-%d"))
sample.plot()
plt.xticks(rotation=60)
plt.xticks(tick)

In [ ]:
# sns.barplot(x='Description', y='total_spend', data=df_temp)

In [ ]:
df_temp.groupby(['Description'])['total_spend'].sum().sort_values(ascending=False).head(10).plot(kind='barh')

In [ ]:
# print(str(df_temp.iloc[541904]['InvoiceDate']).split(" ")[0])
# print(pd.to_datetime(str(df_temp.iloc[541904]['InvoiceDate']).split(" ")[0], format="%Y-%m-%d"))

# print(str(df_temp.iloc[541904]['InvoiceDate']).split(" ")[1])
# print(pd.to_datetime(str(df_temp.iloc[541904]['InvoiceDate']).split(" ")[1], format="%H:%M:%S"))

# print(df_temp['InvoiceDate'].dt.hour)

In [ ]:
df_temp['hour'] = pd.to_datetime(df_temp['InvoiceDate']).dt.hour
df_temp.head()

In [ ]:
df_temp.groupby('Country')["InvoiceNo"].count().sort_values(ascending=False).head(20)

In [ ]:
df_temp.groupby('Country')["total_spend"].sum().sort_values(ascending=False).head(20).sort_values(ascending=True).plot(kind='barh')

In [ ]:
df_temp['hour'].value_counts().sort_index().plot(kind='bar')

In [ ]:
country = "United Kingdom"
df_temp.loc[df_temp['Country'] == f"{country}"].groupby('hour')['total_spend'].sum().plot(kind='bar', color='darkblue')
plt.title(f"Total Spend by Hour in {country}")

In [ ]:
df_temp.describe()

In [ ]:
df_temp.loc[df_temp['Quantity'] >= 60_000]

In [ ]:
df_temp.loc[df_temp['UnitPrice'] >= 30_000]

In [ ]:
df_temp['StockCode'] = df_temp['StockCode'].str.upper()

In [ ]:
df_temp[df_temp['StockCode'] == 'm']

In [ ]:
df_temp[df_temp['StockCode'] == 'M']

In [ ]:
df_temp[df_temp['Description'] == 'Manual']['total_spend'].describe()

In [ ]:
df_temp['Description'].value_counts().tail(20)

In [ ]:
df_temp.loc[df_temp['Description'] == 'historic computer difference?....se']

In [ ]:
df_temp.loc[df_temp['StockCode'] == '84944']

In [ ]:
df_temp.loc[df_temp['Description'] == 'Missing']

In [ ]:
df_temp.loc[df_temp['StockCode'] == '22458'].head()

In [ ]:
df_temp[df_temp['StockCode'].str.contains('90126')]

In [ ]:
df_temp[df_temp['StockCode'].apply(lambda x: len(x) < 5)]['StockCode'].value_counts()

In [ ]:
df_temp[df_temp['StockCode'] == "M"]
# df_temp[df_temp['StockCode'].str.contains('POST')]
# df_temp[df_temp['InvoiceNo'].str.startswith('C')]
# df_temp[df_temp['StockCode'].str.contains('DOT')]['StockCode'].value_counts()
# df_temp[df_temp['StockCode'].str.contains('C2')]['Country'].value_counts()

In [ ]:
df_temp.groupby('StockCode')['Description'].value_counts().head(20)

In [ ]:
df_temp.columns

In [ ]:
most_freq = df_temp.groupby('StockCode')['Description'].agg(lambda x: x.value_counts().idxmax()).reset_index()
df = pd.merge(df_temp, most_freq, on='StockCode', how='left')
df = df.drop(columns=['Description_x'])
df = df.rename(columns={'Description_y': 'Description'})
df = df.reindex(columns=['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'total_spend', 'hour'])

In [ ]:
df_temp.loc[df_temp['Description'] == 'damaged'].head()

In [ ]:
df_temp.loc[df_temp['StockCode'].str.contains("10123")]

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 3))
ax[0].scatter(x=df_temp['Quantity'], y=df_temp['UnitPrice'])
ax[0].title.set_text('Quantity vs UnitPrice')
ax[1].scatter(x=df_temp['Quantity'], y=df_temp['total_spend'])
ax[1].title.set_text('Quantity vs total_spend')
ax[2].scatter(x=df_temp['UnitPrice'], y=df_temp['total_spend'])
ax[2].title.set_text('UnitPrice vs total_spend')

---

In [ ]:
df = pd.read_csv("./data/uncleaned_data.csv", encoding='cp1252')

# Clean the data
# df['Description'] = df['Description'].fillna('No Description')
df['CustomerID'] = df['CustomerID'].fillna(0)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['total_spend'] = df['Quantity'] * df['UnitPrice']

df['InvoiceNo'] = df['InvoiceNo'].astype(str)
df['StockCode'] = df['StockCode'].astype(str)
df['Description'] = df['Description'].astype(str)
df['CustomerID'] = df['CustomerID'].astype(float).astype(int)
df['Country'] = df['Country'].astype(str)
df['total_spend'] = df['total_spend'].astype(float)

df['StockCode'] = df['StockCode'].str.upper()
most_freq = df.groupby('StockCode')['Description'].agg(lambda x: x.value_counts().idxmax()).reset_index()
columns_index = df.columns
df = df.drop(columns=['Description'])
df = pd.merge(df, most_freq, on='StockCode', how='left')
df = df.reindex(columns=columns_index)

# Write the cleaned data to a new csv file
# df.to_parquet("./data/cleaned_data.parquet")
df.to_parquet("./data/cleaned_data.csv")

---